In [ ]:
# from rerank import BGEReranker
# from search_with_bge import QdrantSearchBGE

import pandas as pd
from tqdm import tqdm
from typing import List, Dict, Callable

c:\Users\Admin\miniconda3\envs\crypto_agent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bge_search_instance = QdrantSearchBGE(
        host="http://localhost:6333",
        collection_name= "law_corpus_bge_v2",
        model_name="BAAI/bge-m3",
        use_fp16=True
    )

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 7887.49it/s]


In [3]:
client = bge_search_instance.client
query_test = "Hồ sơ tuyển chọn Thẩm phán Tòa án nhân dân cấp huyện phải được giao lại cho Chánh án Tòa án nhân dân cấp tỉnh quản lý khi nào?"


In [6]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost", port=6333)

count = client.count(collection_name="law_corpus_bge", exact=True).count
print(f"📊 Total points in 'law_corpus_bge_v2': {count}")

📊 Total points in 'law_corpus_bge_v2': 173680


In [121]:
count = client.count(collection_name="law_corpus_bge_v2", exact=True).count
print(f"📊 Total points in 'law_corpus_bge': {count}")

📊 Total points in 'law_corpus_bge': 170000


In [8]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost", port=6333)

collections = client.get_collections().collections

print("📚 Existing collections in Qdrant:\n")
for col in collections:
    count = client.count(collection_name=col.name, exact=True).count
    print(f"• {col.name}: {count} points")

📚 Existing collections in Qdrant:

• chat_questions: 0 points
• law_corpus_openai: 1075 points
• legal_documents_collection: 1 points


C:\Users\Admin\AppData\Local\Temp\ipykernel_9792\2971967807.py:3: UserWarning: Qdrant client version 1.15.0 is incompatible with server version 1.8.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient("http://localhost", port=6333)


In [123]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct

def is_valid_vector(vec, expected_dim=1024):
    if vec is None or len(vec) != expected_dim:
        return False
    if any([v is None or isinstance(v, str) or np.isnan(v) for v in vec]):
        return False
    return True

def migrate_collection(old_collection, new_collection):
    client = QdrantClient("http://localhost", port=6333)

    # Tạo collection mới
    client.recreate_collection(
        collection_name=new_collection,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
    )
    print(f"✅ Created new collection {new_collection}")

    offset = None
    total = 0
    skipped = 0

    while True:
        points, offset = client.scroll(
            collection_name=old_collection,
            limit=500,
            with_vectors=True,
            with_payload=True,
            offset=offset
        )

        converted = []
        for pt in points:
            if not is_valid_vector(pt.vector):
                print(f"⚠️ Skipped: id={pt.id}")
                skipped += 1
                continue
            converted.append(PointStruct(id=pt.id, vector=pt.vector, payload=pt.payload))

        if converted:
            client.upsert(collection_name=new_collection, points=converted)
            print(f"🔁 Migrated {len(converted)} points")
            total += len(converted)

        if offset is None:
            break

    print(f"✅ Migration complete: {total} migrated, {skipped} skipped")


In [ ]:
migrate_collection("law_corpus_bge_v2", "law_corpus_bge_v3")